In [2]:
# import all depedencies first cell
import numpy as np
import pandas as pd
from sklearn import linear_model, metrics, cross_validation

In [3]:
# define functions in one cell, call when you need
def read_csv(path):
    return pd.read_csv(path)

In [4]:
path = '../../assets/datasets/bikeshare.csv'
bikeshare = read_csv(path)

In [5]:
# get the data dictionary of bikeshare to see possible feature candidates
bikeshare.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

In [6]:
# assume we want 2 features, 'temp' and 'hum' and weathersit, because
# weathersit is categorical we must make dummy variables
# create a new dataframe with only features temp and hum, 
# and all dummy variables required for weathersit (hint n - 1 classes)
# hint (pd.get_dummies(df_column)) will transform your variable 
# into n dummy classes
# hint, use .join to merge two dataframes on a common key (default inner join on index)
# call your features dataframe 'modeldata' and your y response column 'y'

modeldata = pd.DataFrame(bikeshare['temp'])
modeldata['hum'] = bikeshare['hum']
dummy_location = pd.get_dummies(bikeshare['weathersit'],prefix='weathersit')
cols_to_keep = ['temp','hum']
modeldata = modeldata[cols_to_keep].join(dummy_location.ix[:, :])
del modeldata['weathersit_1']
y = pd.DataFrame(bikeshare['casual'])
modeldata.head(20)

,temp,hum,weathersit_2,weathersit_3,weathersit_4
0,0.24,0.81,0.0,0.0,0.0
1,0.22,0.80,0.0,0.0,0.0
2,0.22,0.80,0.0,0.0,0.0
3,0.24,0.75,0.0,0.0,0.0
4,0.24,0.75,0.0,0.0,0.0
5,0.24,0.75,1.0,0.0,0.0
6,0.22,0.80,0.0,0.0,0.0
7,0.20,0.86,0.0,0.0,0.0
8,0.24,0.75,0.0,0.0,0.0
9,0.32,0.76,0.0,0.0,0.0


In [9]:
 kf = cross_validation.KFold(len(modeldata), n_folds=5, shuffle=True)
# you must define modeldata in cell above for kf assignment to run
# kf returns a dictionary composed of key-value pairs

In [10]:
lm_obj = linear_model.LinearRegression() # instaniate only one time
scores = [] # we will append mse scores from each iteration in kf
for train_index, test_index in kf: # for (key,value) in dictionary
    x_train = modeldata.iloc[train_index] # get new set each iteration
    y_train = y.iloc[train_index]
    
    x_test = modeldata.iloc[test_index] # get new test data each iteration
    y_test = y.iloc[test_index]
        
    lm = lm_obj.fit(x_train, y_train) # fit new model each iteration
    x_test_pred = lm.predict(x_test)

    mse = metrics.mean_squared_error(y_test,x_test_pred) 
    # get new mse each iteration
    scores.append(mse) # append mse scores from each model to scores list

In [13]:
# print the mean mse score from all iterations, explain output
print np.mean(scores)

1674.18336234


In [16]:
# fit a regression model on all the model and outcome data (modeldata and y)
# hint use, linear_model.LinearRegression().fit()
# get predictions from fitted model using same model data
# calculate MSE and interpret 
# hint MSE = SSE/n = (y - y_est)/n, y_est = lm.predict(modeldata)
X = modeldata
lm = linear_model.LinearRegression().fit(X,y)
predictions = lm.predict(X)

print metrics.mean_squared_error(y,predictions)
#print (y - predictions).sum() / len(predictions)

1672.58110765


In [23]:
# fit a lasso regression model on all the model and outcome data (modeldata and y)
# hint use, linear_model.Lasso().fit()
# get predictions from fitted model using same model data
# calculate MSE and interpret
# hint MSE = SSE/n = (y - y_est)/n, y_est = lm.predict(modeldata)
lm = linear_model.Lasso(alpha = 10.0).fit(X,y)
predictions = lm.predict(X)

print metrics.mean_squared_error(y,predictions)

2430.84614081


In [33]:
# fit a Ridge regression model on all the model and outcome data (modeldata and y)
# hint use, linear_model.Ridge().fit()
# get predictions from fitted model using same model data
# calculate MSE and interpret
# hint MSE = SSE/n = (y - y_est)/n, y_est = lm.predict(modeldata)
lm = linear_model.Ridge(alpha = 10.0).fit(X,y)
predictions = lm.predict(X)

print metrics.mean_squared_error(y,predictions)

1672.82630501
